# Pfizer
## Next Year
* New pdf is not readable. Abbeyy Finereader job...

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import re

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [36]:
# Read pdf into DataFrame
df = pd.read_csv("efpiadisclosurereport2019ch.csv", sep=",")

## Format Table

In [41]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty0', 'name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total']

#Remove empties
df_export.drop(columns=['empty0', 'empty1'], inplace=True)

#Remove empty rows
df_export = df_export.dropna(subset=['name'], how='all')
df_export = df_export.dropna(subset=['country'], how='all')

#Remove by string
df_export = df_export[~df_export['name'].str.contains('Full Name', na=False)]
df_export = df_export[~df_export['name'].str.contains('|% of the nu', na=False, regex=False)]

#Type by text
df_export = set_type_by_alphabetical_order(df_export)

#Revert name
df_export = revert_name(df_export, ',')

#Cleanup Numbers
df_export = replace_in_number(df_export, 'o', '0')
df_export = replace_in_number(df_export, '»', '0')
df_export = replace_in_number(df_export, '«', '0')
df_export = replace_in_number(df_export, '<“', '0')
df_export = replace_in_number(df_export, '•', '0')

df_export = cleanup_number(df_export)
#df_export = amounts_to_number(df_export)

#Add empty fields
df_export = add_plz(df_export)

#Remove Carinations
df_export = remove_carination(df_export, " ")

#basic string conversion
df_export = basic_string_conversion(df_export)

In [42]:
def fix_errors(df_export, field):
    other_fields = number_fields.copy()
    other_fields.remove(field)
    
    #Loop each row
    for index, row in df_export[df_export[field] == 'ERROR'].iterrows():
        
        #Check if other fields are numeric. If not, break
        has_strings = False
        for f in other_fields:
            if re.match(r'^[0-9.]*$', str(row[f])) == None:
                has_strings = True
                break
            
        #if no strings, calculate!
        if not has_strings:
            if field != 'total':
                without_total = other_fields.copy()
                without_total.remove('total')
                df_export.loc[index, field] = (
                    pd.to_numeric(df_export.loc[index, 'total']) 
                    - pd.to_numeric(df_export.loc[index, without_total[0]])
                    - pd.to_numeric(df_export.loc[index, without_total[1]])
                    - pd.to_numeric(df_export.loc[index, without_total[2]])
                    - pd.to_numeric(df_export.loc[index, without_total[3]])
                    - pd.to_numeric(df_export.loc[index, without_total[4]])
                )
            else:
                without_total = other_fields.copy()
                df_export.loc[index, field] = (
                    pd.to_numeric(df_export.loc[index, without_total[0]])
                    + pd.to_numeric(df_export.loc[index, without_total[1]])
                    + pd.to_numeric(df_export.loc[index, without_total[2]])
                    + pd.to_numeric(df_export.loc[index, without_total[3]])
                    + pd.to_numeric(df_export.loc[index, without_total[4]])
                    + pd.to_numeric(df_export.loc[index, without_total[5]])
                )
                           
    return df_export

  
#Fill ERROR
for field in number_fields:
    df_export.loc[df_export[field].str.contains(r'^[0-9.]*$', regex=True) == False, field] = 'ERROR'
   
#Fill na in numeric fields
for f in number_fields:
    df_export[f] = df_export[f].fillna(value=0)

#Fix Errors
for f in number_fields:
    df_export = fix_errors(df_export, f)  

In [43]:
#Export
export_list(df_export, 'pfizer')

donations_grants not a number
sponsorship not a number
registration_fees not a number
travel_accommodation not a number
fees not a number
related_expenses not a number
total not a number
saved


In [29]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx', open=True)